# All Data - NLP EDA - NER on Company Names

(Slow file separated out from other code)

In [1]:
# Flag for if you are running this on the sample dataset
sample = False
# Modify this path as needed to run on your machine
sample_path = r'~\Box\STAT 222 Capstone\Intermediate Data\all_data_sample.csv'

In [2]:
# Packages
import pandas as pd
import matplotlib.pyplot as plt
import os
import time

In [3]:
# Load in sample csv, or full parquet file
if sample:
    df = pd.read_csv(sample_path)
else:
    # list of files in '../../../Data/All_Data/All_Data_Fixed_Quarter_Dates'
    file_list = [f for f in os.listdir(r'../../../Data/All_Data/All_Data_Fixed_Quarter_Dates') if f.endswith('.parquet')]
    # read in all parquet files
    df = pd.concat([pd.read_parquet(r'../../../Data/All_Data/All_Data_Fixed_Quarter_Dates/' + f) for f in file_list])

df

,ticker,fixed_quarter_date,earnings_call_date,Rating,Rating Agency Name,rating_date,Source,Rating Rank AAA is 10,Next Rating,Next Rating Date,...,Change Direction Since Last Fixed Quarter Date,Change Since Last Fixed Quarter Date,Ticker,Description,Company Name,Sector,Industry Group,Industry,Sub-Industry,Comment
0,AAPL,2014-07-01 00:00:00.000000,2014-04-23,AA,Standard & Poor's Ratings Services,2014-05-27,Supplementary,9,AA,2015-02-18,...,None,NaN,AAPL,"Apple Inc. (Apple) designs, manufactures and m...",APPLE INC,Information Technology,Technology Hardware & Equipment,"Technology Hardware, Storage & Peripherals","Technology Hardware, Storage & Peripherals","Manufacturers of cellular phones, personal com..."
1,AAPL,2014-10-01 00:00:00.000000,2014-07-22,AA,Standard & Poor's Ratings Services,2014-05-27,Supplementary,9,AA,2015-02-18,...,Same As Last Fixed Quarter Date,0.0,AAPL,"Apple Inc. (Apple) designs, manufactures and m...",APPLE INC,Information Technology,Technology Hardware & Equipment,"Technology Hardware, Storage & Peripherals","Technology Hardware, Storage & Peripherals","Manufacturers of cellular phones, personal com..."
2,AAPL,2015-01-01 00:00:00.000000,2014-10-20,AA,Standard & Poor's Ratings Services,2014-05-27,Supplementary,9,AA,2015-02-18,...,Same As Last Fixed Quarter Date,0.0,AAPL,"Apple Inc. (Apple) designs, manufactures and m...",APPLE INC,Information Technology,Technology Hardware & Equipment,"Technology Hardware, Storage & Peripherals","Technology Hardware, Storage & Peripherals","Manufacturers of cellular phones, personal com..."
3,AAPL,2015-04-01 00:00:00.000000,2015-01-27,AA,Standard & Poor's Ratings Services,2015-02-18,Supplementary,9,AA,2015-05-28,...,Same As Last Fixed Quarter Date,0.0,AAPL,"Apple Inc. (Apple) designs, manufactures and m...",APPLE INC,Information Technology,Technology Hardware & Equipment,"Technology Hardware, Storage & Peripherals","Technology Hardware, Storage & Peripherals","Manufacturers of cellular phones, personal com..."
4,AAPL,2015-07-01 00:00:00.000000,2015-04-27,AA,Standard & Poor's Ratings Services,2015-06-02,Supplementary,9,AA,2015-08-25,...,Same As Last Fixed Quarter Date,0.0,AAPL,"Apple Inc. (Apple) designs, manufactures and m...",APPLE INC,Information Technology,Technology Hardware & Equipment,"Technology Hardware, Storage & Peripherals","Technology Hardware, Storage & Peripherals","Manufacturers of cellular phones, personal com..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
912,ZTS,2015-10-01 00:00:00.000000,2015-08-04,BBB,Standard & Poor's Ratings Services,2015-01-30,Both,7,BBB,2015-11-03,...,Same As Last Fixed Quarter Date,0.0,ZTS,"Zoetis Inc. is focused on the discovery, devel...",ZOETIS INC,Health Care,"Pharmaceuticals, Biotechnology & Life Sciences",Pharmaceuticals,Pharmaceuticals,"Companies engaged in the research, development..."
913,ZTS,2016-01-01 00:00:00.000000,2015-11-03,BBB,Standard & Poor's Ratings Services,2015-11-03,Supplementary,7,BBB,2016-01-22,...,Same As Last Fixed Quarter Date,0.0,ZTS,"Zoetis Inc. is focused on the discovery, devel...",ZOETIS INC,Health Care,"Pharmaceuticals, Biotechnology & Life Sciences",Pharmaceuticals,Pharmaceuticals,"Companies engaged in the research, development..."
914,ZTS,2016-04-01 00:00:00.000000,2016-02-16,BBB,Standard & Poor's Ratings Services,2016-01-22,Both,7,BBB,2016-12-23,...,Same As Last Fixed Quarter Date,0.0,ZTS,"Zoetis Inc. is focused on the discovery, devel...",ZOETIS INC,Health Care,"Pharmaceuticals, Biotechnology & Life Sciences",Pharmaceuticals,Pharmaceuticals,"Companies engaged in the research, development..."
915,ZTS,2016-07-01 00:00:00.000000,2016-05-04,BBB,Standard & Poor's Ratings Services,2016-01-22,Both,7,BBB,2016-12-23,...,Same As Last Fixed Quarter Date,0.0,ZTS,"Zoetis Inc. is focused on the discovery, devel...",ZOETIS INC,Health Care,"Pharmaceuticals, Biotechnology & Life Sciences",Pharmaceuticals,Pharmaceuticals,"Companies engaged in the research, development.

## NER with Spacy - number of companies


In [12]:
import spacy
from tqdm.auto import tqdm # progress tracking
tqdm.pandas()

# load model and disable unnecessary components
activated = spacy.prefer_gpu()
print('using gpu? ', activated)
nlp = spacy.load('en_core_web_sm', disable=['tagger', 'parser', 'lemmatizer', 'attribute_ruler', 'morphologizer', 'textcat', 'toc2vec'])

using gpu?  False


In [21]:
# Function to process transcript and count company mentions
def count_companies(text):
    # Process the text
    doc=nlp(text)
    # Counter
    count = 0
    # Initialize count
    for entity in doc.ents:
        # We'll select just entities that are tagged organizations (close enough to companies)
        if entity.label_ == "ORG":
            count += 1
    return count

# Test the function on one transcript

# Start timer
start_time = time.time()

print(count_companies(df['transcript'].iloc[0]))

# Record end time
end_time = time.time()

# Print time to process in minutes
print('Estimated Time to process: ', len(df) * (end_time - start_time) / 60)

165
Estimated Time to process:  98.34576349258423


In [ ]:
# Start timer
start_time = time.time()

# Apply the function to the transcript column
# potentially look into ways to do this in parallel or make more efficient...
df['company_mentions'] = df['transcript'].progress_apply(count_companies)

# Record end time
end_time = time.time()

# Print time to process in minutes
print('Time to process: ', (end_time - start_time).seconds / 60)

In [ ]:
# Save parquet file of ticker, fixed_quarter_date, and company_mentions
df[['ticker', 'fixed_quarter_date', 'company_mentions']].to_parquet(r'../../../Data/All_Data/Company_Mentions.parquet')

In [ ]:
# Plot the distribution of company mentions
plt.figure(figsize=(8, 5))
plt.hist(df['company_mentions'], bins=20, color='skyblue', edgecolor='black')
plt.title('Company Mentions Distribution')
plt.xlabel('Number of Company Mentions')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

print('Average number of company mentions:', df['company_mentions'].mean())

In [ ]:
# Testing the nlp model
sample_text = 'Microsoft is a great company. I also like Apple and Google.'
print('testing on sample text:')
print(sample_text)
doc = nlp(sample_text)
for entity in doc.ents:
    print(entity.text, entity.label_)
harder_sample_text = 'I am going to the bank to deposit my money. apple had a great quarter.'
print('testing on harder sample text:')
print(harder_sample_text)
doc = nlp(harder_sample_text)
for entity in doc.ents:
    print(entity.text, entity.label_)